# Data 설정

In [1]:
import torch
import torch.nn as nn
import torchvision
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# MNIST 데이터셋 
train_data = datasets.MNIST(
    root="../data",
    train=True,
    download=True,
    transform=transforms.ToTensor(),
)

test_data = datasets.MNIST(
    root="../data",
    train=False,
    download=True,
    transform=transforms.ToTensor(),
)

# Data loader
train_loader = DataLoader(train_data, batch_size=128, shuffle=True)
test_loader = DataLoader(test_data, batch_size=128, shuffle=False)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw



# Model 설정

In [2]:
class Model(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(Model, self).__init__()
        self.mlp1 = nn.Linear(input_size, hidden_size) 
        self.relu = nn.ReLU()
        self.mlp2 = nn.Linear(hidden_size, num_classes)  
        
    def forward(self, x):
        out = self.mlp1(x)
        out = self.relu(out)
        out = self.mlp2(out)
        
        return out

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  # 여기서 Cuda는 GPU의 일종이라고 보면 된다. 
model = Model(input_size=28*28*1, hidden_size=100, num_classes=10).to(device)

In [4]:
CELoss = nn.CrossEntropyLoss() # loss 함수 설정이고 안에 소프트맥스 안에 있다. 
adam_optimizer = torch.optim.Adam(model.parameters(), lr=0.001) # 최적화 알고리즘 class 선언 설명: model의 파라미터로 아담에 적용, learning rate는 0.001, 0.002, 0.005 학습할때 중요하다. 미리 알지는 못하고 실험하고나서 튜닝과정이 필요하다. 

# 뉴럴 네트워크 모델 학습

In [11]:
# 뉴럴 네트워크 모델 학습
total_epochs = 3 # 세번동안 전체 데이터를 학습 시킨다. 
print('number of iteration :', len(train_loader))
# epoch : 모든 데이터를 한 번 학습하는 단위
for epoch in range(total_epochs):
    # iteration : 한 'mini-batch' 단위의 데이터를 학습하는 단위
    for i, (images, labels) in enumerate(train_loader):  
        # images : [mini-batch, 1, 28, 28]
        # labels : [mini-batch]
        images = images.reshape(-1, 28*28).to(device)   # 멀티레이어에 넣기 위해서 reshape로 이미지를 펴준다. 
        labels = labels.to(device)  # 정답데이터
        
        # Forward pass : 이론시간에 배운 인풋데이터넣고 가중치를 계산 한다. 그것이 모델학습이다. 
        outputs = model(images)
        ce_loss = CELoss(outputs, labels) # model에서 학습된 이미지 데이터와 정답데이터의 손실률을 계산한다.
        
        # Backward and optimize
        adam_optimizer.zero_grad() # 다양한 optimization 기법 적용 가능, adam_optimizer.zero_grad() 는 처음에 그래디언트를 초기화를 해준다. 
        ce_loss.backward() # Back propagation; for문 안에서 에서 손실계산이 되고 난뒤 back propagation이 진행이 된다. 
        adam_optimizer.step() # adam_optimizer 작동이 된다. 
            
    print ('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, total_epochs, ce_loss.item()))  # 여기서 .item()은 하나의 숫자로 표현해주기 위해서 쓴다. 

number of iteration : 469
Epoch [1/3], Loss: 0.0407
Epoch [2/3], Loss: 0.0621
Epoch [3/3], Loss: 0.0709


In [13]:
images = images.reshape(-1, 28*28).to(device) # 디바이스로 사용하도록 명시 
images.shape

torch.Size([16, 784])

In [17]:
# 학습이 끝난 후 모델 성능 테스트
# test에서는 back propagation 작업을 하지 않으므로 gradient를 계산하지 않도록 함 - 메모리의 효율성을 위해
with torch.no_grad(): # gradient 계산하지 않도록 하는 코드
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)  # .data?? 뭐지?? 모델이 뽑은 아웃풋중에 가장 최대값을 predict에 집어 넣어넣겠다는 소리다. ㅡ,: 이표시는 뭐지?
        total += labels.size(0) # 정확도를 계산하기 위해 배치사이즈 크기 만큼 매번 이터레이션을 돌면서 total의 수를 더해주고,  
        correct += (predicted == labels).sum().item() # item() 하나의 숫자로 바꾸기 위해서.

    print('Accuracy of the network on the 10000 test images: {} %'.format(100 * correct / total))  # 퍼센트로 나타내기 위해 100을 곱함. 전체 데이터(경우의 수)분의 맞은 개수 * 100 = 정확도

In [ ]:
# 학습한 모델을 model.ckpt라는 이름으로 현재 경로에 저장
torch.save(model.state_dict(), 'model_basic.ckpt')

# 최적화 함수

In [ ]:
# Stochastic Gradient Descent
sgd_optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

# Stochastic Gradient Descent with momentum
sgd_with_momentum_optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)  # 모멘텀은 얼머나 관성을 줘서 로컬미니멈을 빠져나가게 해준다. 즉 클로벌 미니범에 가까이 가게 해준다. 

# Adagrad
Adagrad_optimizer = torch.optim.Adagrad(model.parameters(), lr=0.01)  # Adagrad는 러닝레이트를 어떻게 어댑티드하게 조절할수 있는지 대한 방법이다.  

# RMSprop 
RMSprop_optimizer = torch.optim.RMSprop(model.parameters(), lr=0.01)  # Adagrad가 학습이 진행됨에 따라 러닝웨이트가 너무 작아질수 잇는 문제 때문에 지수평균으로 대체해서 해결하는 컨셉이다. 

# Adam
adam_optimizer = torch.optim.Adam(model.parameters(), lr=0.001) 

In [ ]:
# 뉴럴 네트워크 모델 학습
total_epochs = 3
for epoch in range(len(train_loader)):
    for i, (images, labels) in enumerate(train_loader):  
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        ce_loss = CELoss(outputs, labels)
        
        # Backward and optimize
        ##sgd_optimizer##.zero_grad()  # ##~## 이부분을 위에 4개중에 골라서 넣으면 된다. 최적화 할떄 
        ce_loss.backward() # Back propagation
        ##sgd_optimizer##.step() # optimizer 작동
            
    print ('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, total_epochs, ce_loss.item()))

# Dropout

코드 사용법: 1. 활성함수 전에 쓴다. 
             2. 뉴럴네트워크(=멀티레이어) 후에 쓴다. 

In [ ]:
# Dropout 추가
class Model(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(Model, self).__init__()
        self.mlp1 = nn.Linear(input_size, hidden_size)
        self.dropout = nn.Dropout(p=0.5) # p : probability of an element to be zeroed. Default: 0.5 ; 각각 요소들이 0이 될 확률이다. 정확히 얼마큼 끊이 줄지 정하는 것이다. 하는 이유는? 레큘러라이제이션을 하기 위해
        self.relu = nn.ReLU()
        self.mlp2 = nn.Linear(hidden_size, num_classes)  
        
    def forward(self, x):
        out = self.mlp1(x)
        out = self.dropout(out) # dropout 추가!
        out = self.relu(out)
        out = self.mlp2(out)
        return out

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Model(input_size=28*28*1, hidden_size=100, num_classes=10).to(device)

# Batch Normalization ??

In [ ]:
# Batch Normalization 추가;  보통 넣는 코드 위치는  뉴럴네트워크 다음 활성함수전에 넣는다. 
class Model(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(Model, self).__init__()
        self.mlp1 = nn.Linear(input_size, hidden_size)
        self.bn = nn.BatchNorm1d(hidden_size) # batch normalization 1d 그리고 2d, 3d도 있다. 지금까지 nn.Linear를 넣기 위해 1d 벡터로 펴주는 작업을 해왔다. output_feature이 크기 히든 사이즈를 넣으면 된다. 
        self.relu = nn.ReLU()
        self.mlp2 = nn.Linear(hidden_size, num_classes)  
        
    def forward(self, x):
        out = self.mlp1(x)
        out = self.dropout(out) # batch normalization 추가!
        out = self.relu(out)
        out = self.mlp2(out)
        return out

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Model(input_size=28*28*1, hidden_size=100, num_classes=10).to(device)